In [1]:
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter("ignore")

In [2]:
up = gpd.read_file(r"F:\DSLR\Folium\Geojson\Upazilla_updated.geojson")
dist = gpd.read_file(r"F:\DSLR\Folium\Geojson\BGD_adm2.geojson")

In [3]:
up_desc = pd.read_csv(r"F:\DSLR\Folium\Animate\describe.csv")
dist_desc = pd.read_csv(r"F:\DSLR\Folium\Animate\District_desc.csv")

In [32]:
def get_div_cen(div_name):
    div_cen = {'Barisal': [22.419147168685168, 90.34719389574566],
                'Chittagong': [22.709657647813373, 91.73176862241395],
                'Dhaka': [23.839712023586152, 90.2416568363619],
                'Khulna': [22.917754757675876, 89.29251999630705],
                'Mymensingh': [24.846766716747663, 90.38045980128486],
                'Rajshahi': [24.588748960942787, 89.04457003840928],
                'Rangpur': [25.779405192595792, 89.05583622451685],
                'Sylhet': [24.71545553158574, 91.66406548888463]}
    return div_cen[div_name]

def df_process(desc_df, dist_geo, up_geo, div, dist):
    if div != "All":
        if dist == "All":
            sort_df = desc_df[desc_df.Division == div].reset_index(drop = True)
            sort_geo = dist_geo[dist_geo.ADM1_EN == div].reset_index(drop = True)
            centroid = get_div_cen(div)
            zoom = 8
        else:
            sort_df = desc_df[desc_df.District == dist].reset_index(drop = True)
            sort_geo = up_geo[up_geo.ADM2_EN == dist].reset_index(drop = True)
            center = dist_geo[dist_geo.ADM2_EN == dist].reset_index(drop = True).geometry[0].centroid
            centroid = [center.y, center.x]
            zoom = 9
    else:
        sort_df = desc_df
        sort_geo = dist_geo[dist_geo.ADM2_EN.isin(sort_df.District)].reset_index(drop = True)
        centroid = [23.8103, 90.4125]
        zoom = 7

    return sort_df, sort_geo, centroid, zoom


def style_function(fill_color,fill_opacity):
    return lambda feature: {
        'color': "green",
        'weight': 2,
        'fillColor': fill_color,
        'fillOpacity': fill_opacity
    }

def get_op(f_num = 0, f_max = 0, f_min = 0):
    
    op_min = 0.3
    op_max = 0.9
    
    final_op = op_min + ((f_num -f_min)*(op_max - op_min)/(f_max-f_min))
    return final_op

def get_rad(loan, l_max, l_min, dist):
    
    if dist == 'All':
        px_min = 7000
        px_max = 25000
    else:
        px_min = 1000
        px_max = 10000
    
    final_rad = int(px_min + ((loan -l_min)*(px_max - px_min)/(l_max-l_min)))
    return final_rad

def circle_plot(centr, param, l_max, l_min, clr, clr_2, opacity_input, tooltip, dist):
    
    shapesLayer = folium.FeatureGroup(name="Revenue")

    rad = get_rad(param, l_max, l_min, dist)
    
    folium.Circle(location=[centr.y, centr.x],
                  radius= rad,
                  color=clr,
                  weight=2,
                  fill_color=clr_2,
                  fill_opacity= opacity_input,
                  tooltip = tooltip,
                  stroke = True).add_to(shapesLayer)
    return shapesLayer

def get_tt(toggle, upz, area, fh, crp, tca, crp_area, prcnt, dist):
    
    if toggle == False:
        if crp != "Cultivated_area":
            tooltip_html = """
            <div style="font-size: 14px">
            <b>District: </b> {upz}
            <br>
            <b>Area: </b> {area} acre
            <br>
            <b>Firm Holdings: </b> {fh}
            <br>
            <b>Crop Area: </b> {tca} acre
            <br>
            <b>{crp}: </b> {crp_area} acre ({prcnt}%)
            </div>
            """.format(upz=upz, area=area, fh=fh, crp=crp, tca=tca, crp_area=crp_area, prcnt=prcnt)
        else:
            tooltip_html = """
            <div style="font-size: 14px">
            <b>District: </b> {upz}
            <br>
            <b>Area: </b> {area} acre
            <br>
            <b>Firm Holdings: </b> {fh}
            <br>
            <b>Crop Area: </b> {tca} acre
            <br>
            <b>{crp}: </b> {crp_area} acre
            </div>
            """.format(upz=upz, area=area, fh=fh, crp=crp, tca=tca, crp_area=crp_area, prcnt=prcnt)
    else:
        tooltip_html = """
            <div style="font-size: 14px">
            <b>District: </b> {upz}
            <br>
            <b>Area: </b> {area} acre
            <br>
            <b>Firm Holdings: </b> {fh}
            <br>
            <b>{crp} count: </b> {crp_area} ({prcnt}%)
            </div>
            """.format(upz=upz, area=area, fh=fh, crp=crp, crp_area=crp_area, prcnt=prcnt)
        
    if dist != 'All':
        tooltip_html = tooltip_html.replace("District", "Upazilla")

    return tooltip_html

def get_legend(crop, circle_color):
    
        legendHtml_1 = '''
            <div style="position: fixed; 
            bottom: 50px; left: 50px; width: 150px; height: 70px; 
            border:2px solid grey; z-index:9999; font-size:14px; background-color: white; display: flex};
            ">
            &nbsp; Description <br>
            &nbsp; <i class="fa fa-circle" style="color:red"></i> &nbsp; Wind<br>
            </div>
            '''
        return legendHtml_1.replace("Wind", crop).replace("red", circle_color)

In [35]:
def init(life_type, dist_df, dist_geo, up_df, up_geo, div, dist, crop, top_percent):
    
    if (life_type == 'Livestocks'):
        toggle = True
    else:
        toggle = False
    
    if dist == 'All':
        dataframe = dist_df
    else:
        dataframe = up_df
        
    circle_color = 'red'
    if crop != "All":
        op_min = dataframe[crop].min()
        op_max = dataframe[crop].max()
        rad_min = dataframe[crop].min()**0.5
        rad_max = dataframe[crop].max()**0.5
        dataframe = dataframe[dataframe[crop] > dataframe[crop].quantile(1-top_percent)].reset_index(drop = True)
        tt_crp = crop
    else:
        tt_crp = "Cultivated_area"
        rad_min = dataframe.Cultivated_area.min()**0.5
        rad_max = dataframe.Cultivated_area.max()**0.5
        op_min = dataframe.Cultivated_area.min()
        op_max = dataframe.Cultivated_area.max()
        dataframe = dataframe[dataframe.Cultivated_area > dataframe.Cultivated_area.quantile(1-top_percent)].reset_index(drop = True)

    sort_df,sort_geo,centroid,zoom = df_process(dataframe, dist_geo, up_geo, div, dist)
        
    mapObj_dist = folium.Map(location=centroid, zoom_start=zoom, attr='Google', tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}')
    
    if dist != 'All':
        temp_df = dist_geo[dist_geo.ADM2_EN == dist].reset_index(drop=True).iloc[[0]]
        folium.GeoJson(
                    temp_df,
                    style_function=lambda x: {'fillColor': 'green', 'fillOpacity' : 0.03}
                ).add_to(mapObj_dist)
        
    for i in range(len(sort_df)):
        if dist == 'All':
            temp_df = sort_geo[sort_geo.ADM2_PCODE == sort_df.Dist_ID[i]].reset_index(drop=True).iloc[[0]]
            tooltip = folium.Tooltip(get_tt(toggle, sort_df.District[i], f"{int(int(sort_df.area[i])):,}",
                                           f"{int(sort_df.Total_firm_holdings_no[i]):,}",
                                           tt_crp, f"{int(sort_df.Temporary_crops_net_area[i]):,}",
                                            f"{int(sort_df[tt_crp][i]):,}",
                                           int(round((sort_df[tt_crp][i]/sort_df.Temporary_crops_net_area[i]*100),0)), dist))
        else:
            temp_df = sort_geo[sort_geo.ADM3_EN == sort_df.Upazilla[i]].reset_index(drop=True).iloc[[0]]
            tooltip = folium.Tooltip(get_tt(toggle, sort_df.Upazilla[i], f"{int(int(sort_df.area[i])):,}",
                                           f"{int(sort_df.Total_firm_holdings_no[i]):,}",
                                           tt_crp, f"{int(sort_df.Temporary_crops_net_area[i]):,}",
                                            f"{int(sort_df[tt_crp][i]):,}",
                                           int(round((sort_df[tt_crp][i]/sort_df.Temporary_crops_net_area[i]*100),0)), dist))
        op = get_op(sort_df[tt_crp][i], op_max, op_min)


        centr = temp_df.geometry[0].centroid
        shapesLayer = circle_plot(centr, sort_df[tt_crp][i]**0.5, rad_max, rad_min,
                                  circle_color,circle_color, op, tooltip, dist)
        shapesLayer.add_to(mapObj_dist)
        
    legendHtml = get_legend(crop, circle_color)
    mapObj_dist.get_root().html.add_child(folium.Element(legendHtml))
    
    return mapObj_dist

In [36]:
m = init('Livestocks', dist_desc, dist, up_desc, up, "Rangpur", "Dinajpur", "All Fruits", 1)

m

TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'float'

In [46]:
up_desc.dtypes

Division                       object
District                       object
Upazilla_ID                    object
Upazilla                       object
area                            int64
All_holdings                    int64
Non-Firm\nholdings              int64
Total_firm_holdings_no          int64
Small_firm_holdings_no          int64
Medium_firm_holdings_no         int64
Largefirm_holdings_no           int64
Owned_holdings_no               int64
Owned+\ntenant holdings no      int64
Tenant_holdings_no              int64
Agri_labour_households          int64
Fisheries_households            int64
Owned_area                      int64
Operate_area                    int64
Homestead_area                  int64
Cultivated_area                 int64
Temporary_crops_net_area        int64
Irrigated_area                  int64
Temporary_crops_gross_area      int64
Intensity_of_cropping\n(%)      int64
Aus                           float64
Aman                          float64
Boro        

In [7]:
# dist_desc = district_df(updated)
# up_desc = same as before
# dist, up = geojsons
# diision, district, 
# crop name('Aus','Aman', 'Boro', 'Wheat', 'Maize', 'Jute', 'Pulse', 'Oil Seed',
#        'Sugarcane', 'Potato', 'Onion', 'Mustard', 'Turmeric', 'Chili') in case of district level

# in upazilla level 'Onion', 'Mustard', 'Turmeric', 'Chili' are not available for now. rest will work

# final input is to show top k percent

In [45]:
up_desc["All Fruits"].value_counts()

470.78      4
374.64      2
86.97       2
220.23      2
3,435.24    2
           ..
264.8       1
369.18      1
2,219.42    1
642.72      1
426.17      1
Name: All Fruits, Length: 473, dtype: int64